<a href="https://colab.research.google.com/github/hirokimituya/stock-price-analysis/blob/main/technical_analysis/%E9%8A%98%E6%9F%84%E9%81%B8%E5%AE%9A%E9%96%A2%E6%95%B0_csv%E4%BF%9D%E5%AD%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Google Colaboratoryに画像を保存できるように設定
import os
from google.colab import drive
drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive/株式投資')

Mounted at /content/drive


In [2]:
# Ta-Libの代替としてpandas_taを利用している。Ta-Libの関数名を小文字にしたものがあるイメージ
!pip install pandas_ta

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pandas_ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218908 sha256=bf8a9d9e15e6ac11b1ff9feacc509fa3dc7f66b2a6de7ad67e18a551b869f66e
  Stored in directory: /root/.cache/pip/wheels/69/00/ac/f7fa862c34b0e2ef320175100c233377b4c558944f12474cf0
Successfully built pandas_ta


In [3]:
!pip install yahoo_finance_api2

  Preparing metadata (setup.py) ... done
  Created wheel for yahoo_finance_api2: filename=yahoo_finance_api2-0.0.12-py3-none-any.whl size=3712 sha256=3cf4a723bbc21412d19e954eea46ab5340e9bb859bda83d55e7b962eade4171c
  Stored in directory: /root/.cache/pip/wheels/e2/40/e3/f3d8054a41d7d85065555037525f48f56119cafbc5fcddf427
Successfully built yahoo_finance_api2


# 銘柄コードから株価情報を取得する関数

In [4]:
from yahoo_finance_api2 import share as yapi2
from yahoo_finance_api2.exceptions import YahooFinanceError
import datetime as dt
from datetime import date,timedelta
import pandas as pd

# 株価データを取得するメソッド
def get_stock_data_yapi2(code, period_type='year', period=10):
    """株価データを取得する

    :param code: 取得する株価データの銘柄コード
    :param period_type: 取得する株価データの期間の単位（例: 'year', 'month', 'week', 'day'）
    :param period: 取得する株価データの期間の数値
    :return: 株価データのデータフレーム（もし株価データが存在しない場合は、None）
    """
    if period_type == 'month':
        period_type_value = yapi2.PERIOD_TYPE_MONTH
    elif period_type == 'week':
        period_type_value = yapi2.PERIOD_TYPE_WEEK
    elif period_type == 'day':
        period_type_value = yapi2.PERIOD_TYPE_DAY
    else:
        period_type_value = yapi2.PERIOD_TYPE_YEAR

    try:
        data = yapi2.Share(f'{code}.T').get_historical(
            period_type_value,
            period,
            yapi2.FREQUENCY_TYPE_DAY,
            1
        )
    except YahooFinanceError as e:
        print(e.message)
        print(f'銘柄コード[{code}]の株価情報を取得できなかったため、スキップしました。')
        return None

    df = pd.DataFrame(data)

    try:
        # タイムスタンプをDateTime型に変更
        df['datetime'] = pd.to_datetime(df['timestamp'], unit='ms')
    except KeyError as e:
        print(e)
        print(f'銘柄コード[{code}]の株価情報を取得できなかったため、スキップしました。')
        return None

    # 日本時間へ変換
    df['Date'] = df['datetime'] + dt.timedelta(hours=9)

    # インデックスをDateカラムに設定
    df.index = pd.to_datetime(df['Date'].dt.date)

    # 不要なカラムを削除
    del df['Date'], df['datetime'], df['timestamp']

    # 各カラムの最初の文字を大文字に変換
    for col in df.columns:
        df[f'{col.capitalize()}'] = df[col]
        del df[col]

    return df

# 東証上場の銘柄一覧を取得する関数（ダウンロードはしない）

In [5]:
from pandas.io.formats.info import DataFrameTableBuilderNonVerbose
import requests
import pandas as pd

def get_ticker_tse(segment='prime'):
    """東証上場の銘柄コード一覧を取得する

    :param segment: 東証上場の市場区分（例: 'prime' or 'standard' or 'grows'）
    :return: 東証上場の銘柄コード一覧
    """
    segments = {
        'prime': 'プライム',
        'standard': 'スタンダード',
        'grows': 'グロース'
    }

    if segment not in segments:
        raise Exception("get_ticker_tse関数のsegment引数の値は'prime', 'standard', 'grows'のいずれかにしてください")

    # サイトからダウンロードしないようにコメントアウト
    #url = "https://www.jpx.co.jp/markets/statistics-equities/misc/tvdivq0000001vg2-att/data_j.xls"
    #r = requests.get(url)
    #with open('data_j.xls', 'wb') as output:
    #    output.write(r.content)

    df = pd.read_excel("./data/data_j.xls")

    # 引数の市場区分のみに絞り込み
    df = df[df['市場・商品区分'].str.startswith(segments[segment])]

    # カラムの削除とカラム名変更
    df = df[['銘柄名', 'コード', '33業種区分']]   # 必要なカラムのみにする
    df = df.rename(columns={'33業種区分': '業種'})
    df['コード'] = df['コード'].astype(int) # コードを整数に変更

    return df

# 株価情報を出来高や株価、パーフェクトオーダーでフィルタリングする関数

In [6]:
import pandas_ta as ta
import math

def is_perfect_order(stock_df, mas, period_type='D', reverse=False):
    """株価情報がパーフェクトオーダーどうかの判定

    :param stock_df: 対象の株価情報のデータフレーム
    :param mas: 移動平均線に使用するためのリスト（例: [5, 25, 75]）
    :param period_type: 期間をどの単位にするか（'W' or 'M'）
    :param reverse: 下降トレンドのパーフェクトオーダーを検出する場合は、True / 上昇トレンドの場合は、False
    :return: 株価情報がパーフェクトオーダーの場合は、True / それ以外の場合は、False
    """

    df = stock_df.copy()

    # 月足、周足にリサンプリング
    if period_type in ('M', 'W'):
        resampled = df.resample(period_type)
        df = resampled.aggregate({'Open': 'first', 'High': 'max', 'Low': 'min', 'Close': 'last', 'Volume': 'sum'})

    close = df['Close']

    # 移動平均線の算出
    for ma in mas:
        df[f'ma{ma}{period_type}'] = ta.sma(close, ma)

    # パーフェクトオーダーかどうかの判定
    conditions = []
    try:
        if not reverse:
            # 上昇トレンド
            for i in range(len(mas) - 1):
                conditions.append(math.ceil(df[f'ma{mas[i]}{period_type}'].values[-1]) >= math.ceil(df[f'ma{mas[i + 1]}{period_type}'].values[-1]))
        else:
            # 下降トレンド
            for i in range(len(mas) - 1):
                conditions.append(math.ceil(df[f'ma{mas[i]}{period_type}'].values[-1]) <= math.ceil(df[f'ma{mas[i + 1]}{period_type}'].values[-1]))
    except:
        # 株価情報が少なくて移動平均線が取得できない場合は、無条件でFalseでリターンする
        return False


    result = all(conditions)

    return result

In [7]:
def filtered_ticker(df, min_volume=None, max_price=None, min_price=None, perfect_order={}):
    """データフレームの銘柄コードから株価情報を取得して、指定された出来高や株価でフィルタリングする

    :param df: 対象のデータフレーム
    :param min_volume: 出来高の最小値を指定
    :param max_price: 株価の最大値を指定
    :param min_price: 株価の最小値を指定
    :param perfect_order: パーフェクトオーダーの判定に使用する辞書（例: {'mas': [5, 20, 60], 'period_type': 'D', 'reverse': False}）（辞書のキー'mas'のみ必須）
    :return: 出来高や株価でフィルタリングしたデータフレーム
    """
    result_list = []

    # データフレームを辞書のリストに変更
    df_to_dicts = df.to_dict(orient='records')

    # 各銘柄についてループ
    for row in df_to_dicts:
        # コードを取得
        code = row['コード']

        # 株価情報を取得
        if not perfect_order:
            stock_data = get_stock_data_yapi2(code, period_type='day', period=2)
        else:
            largest_mas = perfect_order.get('mas')[-1]
            period_type = perfect_order.get('period_type', 'D')
            if period_type == 'D':
                period = math.ceil(largest_mas / 15)
                stock_data = get_stock_data_yapi2(code, period_type='month', period=period)
            elif period_type == 'W':
                period = math.ceil(largest_mas * 7 / 365)
                stock_data = get_stock_data_yapi2(code, period_type='year', period=period)
            elif period_type == 'M':
                period = math.ceil(largest_mas / 10)
                stock_data = get_stock_data_yapi2(code, period_type='year', period=period)
            else:
                raise ValueError('引数perfect_orderは"D"か"W"か"M"のいずれかを指定してください')
        # 株価情報が取得できなかったら、次の銘柄コードに遷移
        if stock_data is None:
            continue

        # 出来高の条件に合わない場合、次の銘柄コードに遷移
        volume = stock_data['Volume'][-1]
        if min_volume is not None and volume < min_volume:
            continue

        # 価格の条件に合わない場合、次の銘柄コードに遷移
        price = stock_data['Close'][-1]
        if max_price is not None and price > max_price:
            continue
        if min_price is not None and price < min_price:
            continue

        # パーフェクトオーダーの条件に合わない場合、次の銘柄コードに遷移
        if perfect_order:
            mas = perfect_order.get('mas')
            period_type = perfect_order.get('period_type', 'D')
            reverse = perfect_order.get('reverse', False)
            if not is_perfect_order(stock_data, mas=mas, period_type=period_type, reverse=reverse):
                continue

        # 銘柄名と業種を設定
        result_list.append({
            '銘柄名': row['銘柄名'],
            'コード': code,
            '業種': row['業種'],
            '出来高': volume,
            '株価': price,
        })

    # 辞書のリストをデータフレームに変換
    result_df = pd.DataFrame.from_dict(result_list)

    return result_df

# CSVに保存する関数

In [8]:
import datetime as dt
import os
import pytz

jst = pytz.timezone('Asia/Tokyo')
now = dt.datetime.now(jst)
now = now.strftime("%Y%m%d_%H%M%S")
# ディレクトリ名作成（YYYYMMDD_HHMMSS_screening）
directory = f'{now}_screening'

def save_df_as_csv(df, filename):
    """データフレームをCSVファイルとして保存する

    :param df: 対象のデータフレーム
    :param filename: 保存するcsvファイル名
    """
    # もしディレクトリが存在しなければ、作成する
    if not os.path.exists(directory):
        os.makedirs(directory)

    # データフレームをCSVに保存する
    df.to_csv(f'{directory}/{filename}.csv', index=False)

# 東証プライムをフィルタリングしてCSVとして保存

In [9]:
# 東証プライム市場の銘柄コードを取得
tse_prime = get_ticker_tse('prime')
tse_prime

/usr/local/lib/python3.10/dist-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


,銘柄名,コード,業種
0,極洋,1301,水産・農林業
16,ニッスイ,1332,水産・農林業
17,マルハニチロ,1333,水産・農林業
33,雪国まいたけ,1375,水産・農林業
35,サカタのタネ,1377,水産・農林業
...,...,...,...
4262,サンドラッグ,9989,小売業
4263,サックスバー ホールディングス,9990,小売業
4264,ジェコス,9991,卸売業
4267,グローセル,9995,卸売業


In [10]:
# 東証プライムの銘柄コードを以下の条件に合致するもののみにフィルタリング
# 出来高　... 50万株以上
# 株価 ... １００円以上、5000円以下
import time
start_time = time.time()
filtered_tse_prime = filtered_ticker(tse_prime, min_volume=500000, min_price=100, max_price=5000)
print(f'経過時間: {time.time() - start_time}秒')
filtered_tse_prime

Not Found: No data found, symbol may be delisted
銘柄コード[25935]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[4708]の株価情報を取得できなかったため、スキップしました。
Not Found: No data found, symbol may be delisted
銘柄コード[5128]の株価情報を取得できなかったため、スキップしました。
Not Found: No data found, symbol may be delisted
銘柄コード[6924]の株価情報を取得できなかったため、スキップしました。
Not Found: No data found, symbol may be delisted
銘柄コード[8096]の株価情報を取得できなかったため、スキップしました。
経過時間: 454.02580308914185秒


,銘柄名,コード,業種,出来高,株価
0,ニッスイ,1332,水産・農林業,1645100,760.900024
1,ＩＮＰＥＸ,1605,鉱業,8936100,1948.500000
2,大林組,1802,建設業,1669300,1267.500000
3,清水建設,1803,建設業,1633900,968.500000
4,長谷工コーポレーション,1808,建設業,889400,1742.500000
...,...,...,...,...,...
317,ＳＣＳＫ,9719,情報・通信業,615500,2414.000000
318,ヤマダホールディングス,9831,小売業,4734500,449.399994
319,吉野家ホールディングス,9861,小売業,658800,2746.500000
320,ミスミグループ本社,9962,卸売業,767300,2453.500000


In [11]:
# 株価情報が上昇トレンドのパーフェクトオーダー（週足）かどうかの判断
filtered_perfect_order_week_tse_prime = filtered_ticker(filtered_tse_prime, perfect_order={'mas': [5, 20], 'period_type': 'W'})
filtered_perfect_order_week_tse_prime

,銘柄名,コード,業種,出来高,株価
0,ニッスイ,1332,水産・農林業,1645100,760.900024
1,ＩＮＰＥＸ,1605,鉱業,8936100,1948.500000
2,大林組,1802,建設業,1669300,1267.500000
3,清水建設,1803,建設業,1633900,968.500000
4,長谷工コーポレーション,1808,建設業,889400,1742.500000
...,...,...,...,...,...
214,大阪瓦斯,9532,電気・ガス業,1355400,2230.500000
215,エイチ・アイ・エス,9603,サービス業,889700,2013.000000
216,ＳＣＳＫ,9719,情報・通信業,615500,2414.000000
217,吉野家ホールディングス,9861,小売業,658800,2746.500000


In [12]:
# 株価情報が上昇トレンドのパーフェクトオーダー（日足）かどうかの判断
filtered_perfect_order_week_day_tse_prime = filtered_ticker(filtered_perfect_order_week_tse_prime,  perfect_order={'mas': [5, 20, 60]})
filtered_perfect_order_week_day_tse_prime

,銘柄名,コード,業種,出来高,株価
0,ニッスイ,1332,水産・農林業,1645100,760.900024
1,ＩＮＰＥＸ,1605,鉱業,8936100,1948.500000
2,清水建設,1803,建設業,1633900,968.500000
3,鹿島建設,1812,建設業,1405100,2300.000000
4,三井住友建設,1821,建設業,960700,390.000000
...,...,...,...,...,...
108,大阪瓦斯,9532,電気・ガス業,1355400,2230.500000
109,エイチ・アイ・エス,9603,サービス業,889700,2013.000000
110,ＳＣＳＫ,9719,情報・通信業,615500,2414.000000
111,吉野家ホールディングス,9861,小売業,658800,2746.500000


In [13]:
# CSVファイル保存
save_df_as_csv(filtered_perfect_order_week_day_tse_prime, '1_東証P_UPO')

In [14]:
# 株価情報が下降トレンドのパーフェクトオーダー（週足）かどうかの判断
filtered_reverse_perfect_order_week_tse_prime = filtered_ticker(filtered_tse_prime, perfect_order={'mas': [5, 20], 'period_type': 'W', 'reverse': True})
filtered_reverse_perfect_order_week_tse_prime

,銘柄名,コード,業種,出来高,株価
0,日本Ｍ＆Ａセンターホールディングス,2127,サービス業,5731100,733.700012
1,エス・エム・エス,2175,サービス業,558400,2682.500000
2,パーソルホールディングス,2181,サービス業,693100,2462.000000
3,システナ,2317,情報・通信業,1745800,255.000000
4,日本駐車場開発,2353,不動産業,2565700,211.000000
...,...,...,...,...,...
98,イーレックス,9517,電気・ガス業,599300,956.000000
99,レノバ,9519,電気・ガス業,759300,1354.000000
100,エヌ・ティ・ティ・データ,9613,情報・通信業,1732500,1846.000000
101,ヤマダホールディングス,9831,小売業,4734500,449.399994


In [15]:
# 株価情報が下降トレンドのパーフェクトオーダー（日足）かどうかの判断
filtered_reverse_perfect_order_week_day_tse_prime = filtered_ticker(filtered_reverse_perfect_order_week_tse_prime, perfect_order={'mas': [5, 20, 60], 'reverse': True})
filtered_reverse_perfect_order_week_day_tse_prime

,銘柄名,コード,業種,出来高,株価
0,日本Ｍ＆Ａセンターホールディングス,2127,サービス業,5731100,733.700012
1,エス・エム・エス,2175,サービス業,558400,2682.500000
2,システナ,2317,情報・通信業,1745800,255.000000
3,カカクコム,2371,サービス業,802000,1730.000000
4,ベネフィット・ワン,2412,サービス業,691300,1127.000000
...,...,...,...,...,...
70,ＫＤＤＩ,9433,情報・通信業,2647900,4178.000000
71,イーレックス,9517,電気・ガス業,599300,956.000000
72,レノバ,9519,電気・ガス業,759300,1354.000000
73,エヌ・ティ・ティ・データ,9613,情報・通信業,1732500,1846.000000


In [16]:
# CSVファイル保存
save_df_as_csv(filtered_reverse_perfect_order_week_day_tse_prime, '4_東証P_DPO')

# 東証スタンダードをフィルタリングしてCSVとして保存

In [17]:
# 東証スタンダード市場の銘柄コードを取得
tse_standard = get_ticker_tse('standard')
tse_standard

/usr/local/lib/python3.10/dist-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


,銘柄名,コード,業種
34,カネコ種苗,1376,水産・農林業
37,秋川牧園,1380,水産・農林業
38,アクシーズ,1381,水産・農林業
39,ホーブ,1382,水産・農林業
40,ベルグアース,1383,水産・農林業
...,...,...,...
4257,タキヒヨー,9982,卸売業
4260,蔵王産業,9986,卸売業
4265,ヤマザワ,9993,小売業
4266,やまや,9994,小売業


In [18]:
# 東証スタンダードの銘柄コードを以下の条件に合致するもののみにフィルタリング
# 出来高　... 20万株以上
# 株価 ... 100円以上、5000円以下
filtered_tse_standard = filtered_ticker(tse_standard, min_volume=200000, min_price=100, max_price=5000)
filtered_tse_standard

Not Found: No data found, symbol may be delisted
銘柄コード[2777]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[4327]の株価情報を取得できなかったため、スキップしました。
Not Found: No data found, symbol may be delisted
銘柄コード[5104]の株価情報を取得できなかったため、スキップしました。
Not Found: No data found, symbol may be delisted
銘柄コード[5999]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[6032]の株価情報を取得できなかったため、スキップしました。
Not Found: No data found, symbol may be delisted
銘柄コード[7961]の株価情報を取得できなかったため、スキップしました。
Not Found: No data found, symbol may be delisted
銘柄コード[8521]の株価情報を取得できなかったため、スキップしました。
Not Found: No data found, symbol may be delisted
銘柄コード[9318]の株価情報を取得できなかったため、スキップしました。
Not Found: No data found, symbol may be delisted
銘柄コード[9734]の株価情報を取得できなかったため、スキップしました。


,銘柄名,コード,業種,出来高,株価
0,ウエストホールディングス,1407,建設業,429900,2352.000000
1,住石ホールディングス,1514,鉱業,571800,341.000000
2,クルーズ,2138,情報・通信業,2412600,944.000000
3,ｎｍｓ ホールディングス,2162,サービス業,546500,394.000000
4,シダー,2435,サービス業,269100,255.000000
...,...,...,...,...,...
72,Ｊトラスト,8508,その他金融業,437700,424.000000
73,アコム,8572,その他金融業,939100,335.200012
74,トレイダーズホールディングス,8704,証券、商品先物取引業,1696700,723.000000
75,明治海運,9115,海運業,313400,733.000000


In [19]:
# 株価情報が上昇トレンドのパーフェクトオーダー（週足）かどうかの判断
filtered_perfect_order_week_tse_standard = filtered_ticker(filtered_tse_standard, perfect_order={'mas': [5, 20], 'period_type': 'W'})
filtered_perfect_order_week_tse_standard

,銘柄名,コード,業種,出来高,株価
0,クルーズ,2138,情報・通信業,2412600,944.000000
1,シダー,2435,サービス業,269100,255.000000
2,Ｓｈｉｎｗａ Ｗｉｓｅ Ｈｏｌｄｉｎｇｓ,2437,サービス業,260800,570.000000
3,出前館,2484,情報・通信業,754100,403.000000
4,久世,2708,卸売業,213900,1570.000000
5,ＳＡＮＫＯ ＭＡＲＫＥＴＩＮＧ ＦＯＯＤＳ,2762,小売業,232400,163.000000
6,アップルインターナショナル,2788,卸売業,1196900,459.000000
7,焼津水産化学工業,2812,食料品,255400,1140.000000
8,デルソーレ,2876,食料品,363200,484.000000
9,ポラリス・ホールディングス,3010,サービス業,961100,175.000000


In [20]:
# 株価情報が上昇トレンドのパーフェクトオーダー（日足）かどうかの判断
filtered_perfect_order_week_day_tse_standard = filtered_ticker(filtered_perfect_order_week_tse_standard,  perfect_order={'mas': [5, 20, 60]})
filtered_perfect_order_week_day_tse_standard

,銘柄名,コード,業種,出来高,株価
0,クルーズ,2138,情報・通信業,2412600,944.0
1,久世,2708,卸売業,213900,1570.0
2,アップルインターナショナル,2788,卸売業,1196900,459.0
3,焼津水産化学工業,2812,食料品,255400,1140.0
4,デルソーレ,2876,食料品,363200,484.0
5,ポラリス・ホールディングス,3010,サービス業,961100,175.0
6,ＴＨＥグローバル社,3271,不動産業,650600,380.0
7,フジタコーポレーション,3370,小売業,1686900,360.0
8,山喜,3598,繊維製品,315600,176.0
9,クラウディアホールディングス,3607,繊維製品,434500,905.0


In [21]:
# CSVファイル保存
save_df_as_csv(filtered_perfect_order_week_day_tse_standard, '2_東証S_UPO')

In [22]:
# 株価情報が下降トレンドのパーフェクトオーダー（週足）かどうかの判断
filtered_reverse_perfect_order_week_tse_standard = filtered_ticker(filtered_tse_standard, perfect_order={'mas': [5, 20], 'period_type': 'W', 'reverse': True})
filtered_reverse_perfect_order_week_tse_standard

,銘柄名,コード,業種,出来高,株価
0,ウエストホールディングス,1407,建設業,429900,2352.0
1,住石ホールディングス,1514,鉱業,571800,341.0
2,ｎｍｓ ホールディングス,2162,サービス業,546500,394.0
3,ライフドリンク カンパニー,2585,食料品,307500,3605.0
4,イメージ ワン,2667,卸売業,318200,603.0
5,セリア,2782,小売業,206500,2299.0
6,ＪＦＬＡホールディングス,3069,小売業,409600,201.0
7,ユナイテッド・スーパーマーケット・ホールディングス,3222,小売業,542500,1102.0
8,ジー・スリーホールディングス,3647,情報・通信業,987300,166.0
9,ｅｎｉｓｈ,3667,情報・通信業,496600,251.0


In [23]:
# 株価情報が下降トレンドのパーフェクトオーダー（日足）かどうかの判断
filtered_reverse_perfect_order_week_day_tse_standard = filtered_ticker(filtered_reverse_perfect_order_week_tse_standard,  perfect_order={'mas': [5, 20, 60], 'reverse': True})
filtered_reverse_perfect_order_week_day_tse_standard

,銘柄名,コード,業種,出来高,株価
0,ウエストホールディングス,1407,建設業,429900,2352.0
1,ｎｍｓ ホールディングス,2162,サービス業,546500,394.0
2,イメージ ワン,2667,卸売業,318200,603.0
3,ＪＦＬＡホールディングス,3069,小売業,409600,201.0
4,ジー・スリーホールディングス,3647,情報・通信業,987300,166.0
5,ｅｎｉｓｈ,3667,情報・通信業,496600,251.0
6,ディー・エル・イー,3686,情報・通信業,230400,243.0
7,ブロードバンドタワー,3776,情報・通信業,229700,136.0
8,ガーラ,4777,情報・通信業,204300,350.0
9,ハーモニック・ドライブ・システムズ,6324,機械,225200,3560.0


In [24]:
# CSVファイル保存
save_df_as_csv(filtered_reverse_perfect_order_week_day_tse_standard, '5_東証S_DPO')

# 東証グロースをフィルタリングしてCSVとして保存

In [25]:
# 東証スタンダード市場の銘柄コードを取得
tse_grows = get_ticker_tse('grows')
tse_grows

/usr/local/lib/python3.10/dist-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


,銘柄名,コード,業種
55,ルーデン・ホールディングス,1400,建設業
56,エムビーエス,1401,建設業
65,Ｌｉｂ Ｗｏｒｋ,1431,建設業
70,フィット,1436,建設業
75,ニッソウ,1444,建設業
...,...,...,...
4079,プログリット,9560,サービス業
4080,グラッドキューブ,9561,サービス業
4081,ビジネスコーチ,9562,サービス業
4082,Ａｔｌａｓ Ｔｅｃｈｎｏｌｏｇｉｅｓ,9563,サービス業


In [26]:
# 東証グロースの銘柄コードを以下の条件に合致するもののみにフィルタリング
# 出来高　... 20万株以上
# 株価 ... １００円以上、5000円以下
filtered_tse_grows = filtered_ticker(tse_grows, min_volume=200000, min_price=100, max_price=5000)
filtered_tse_grows

Not Found: No data found, symbol may be delisted
銘柄コード[21649]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[3782]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[6067]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[6172]の株価情報を取得できなかったため、スキップしました。


,銘柄名,コード,業種,出来高,株価
0,ＩＴｂｏｏｋホールディングス,1447,建設業,300200,311.0
1,メルディアＤＣ,1739,建設業,341600,794.0
2,ケアネット,2150,サービス業,365900,915.0
3,ＦＲＯＮＴＥＯ,2158,サービス業,460300,611.0
4,ジーエヌアイグループ,2160,医薬品,5448900,1805.0
...,...,...,...,...,...
93,ＩＮＦＯＲＩＣＨ,9338,サービス業,341800,3430.0
94,リニューアブル・ジャパン,9522,電気・ガス業,3275900,1134.0
95,Ｍ＆Ａ総研ホールディングス,9552,サービス業,569600,3370.0
96,マイクロアド,9553,サービス業,980200,2812.0


In [27]:
# 株価情報が上昇トレンドのパーフェクトオーダー（週足）かどうかの判断
filtered_perfect_order_week_tse_grows = filtered_ticker(filtered_tse_grows, perfect_order={'mas': [5, 20], 'period_type': 'W'})
filtered_perfect_order_week_tse_grows

,銘柄名,コード,業種,出来高,株価
0,ＦＲＯＮＴＥＯ,2158,サービス業,460300,611.0
1,ジーエヌアイグループ,2160,医薬品,5448900,1805.0
2,ウェッジホールディングス,2388,その他金融業,996900,134.0
3,ベースフード,2936,食料品,489900,562.0
4,リネットジャパングループ,3556,小売業,227400,633.0
5,オークファン,3674,情報・通信業,1035600,490.0
6,ホットリンク,3680,情報・通信業,361600,385.0
7,Ａｉｍｉｎｇ,3911,情報・通信業,17011200,416.0
8,シェアリングテクノロジー,3989,情報・通信業,1297200,683.0
9,アララ,4015,情報・通信業,329800,674.0


In [28]:
# 株価情報が上昇トレンドのパーフェクトオーダー（日足）かどうかの判断
filtered_perfect_order_week_day_tse_grows = filtered_ticker(filtered_perfect_order_week_tse_grows,  perfect_order={'mas': [5, 20, 60]})
filtered_perfect_order_week_day_tse_grows

,銘柄名,コード,業種,出来高,株価
0,ジーエヌアイグループ,2160,医薬品,5448900,1805.0
1,ベースフード,2936,食料品,489900,562.0
2,ホットリンク,3680,情報・通信業,361600,385.0
3,Ａｉｍｉｎｇ,3911,情報・通信業,17011200,416.0
4,アララ,4015,情報・通信業,329800,674.0
5,ＨＥＮＮＧＥ,4475,情報・通信業,1240800,996.0
6,ＢＡＳＥ,4477,情報・通信業,7215700,326.0
7,免疫生物研究所,4570,医薬品,2263200,809.0
8,プレミアアンチエイジング,4934,化学,8333400,1391.0
9,ドラフト,5070,建設業,376100,481.0


In [29]:
# CSVファイル保存
save_df_as_csv(filtered_perfect_order_week_day_tse_grows, '3_東証G_UPO')

In [30]:
# 株価情報が下降トレンドのパーフェクトオーダー（週足）かどうかの判断
filtered_reverse_perfect_order_week_tse_grows = filtered_ticker(filtered_tse_grows, perfect_order={'mas': [5, 20], 'period_type': 'W', 'reverse': True})
filtered_reverse_perfect_order_week_tse_grows

,銘柄名,コード,業種,出来高,株価
0,ＩＴｂｏｏｋホールディングス,1447,建設業,300200,311.0
1,メルディアＤＣ,1739,建設業,341600,794.0
2,ケアネット,2150,サービス業,365900,915.0
3,総医研ホールディングス,2385,サービス業,223300,248.0
4,アマナ,2402,サービス業,603200,362.0
5,海帆,3133,小売業,1614200,396.0
6,プラスアルファ・コンサルティング,4071,情報・通信業,811800,2594.0
7,ＥＮＥＣＨＡＮＧＥ,4169,情報・通信業,547000,1040.0
8,ココナラ,4176,情報・通信業,345600,289.0
9,エクサウィザーズ,4259,情報・通信業,1282000,394.0


In [31]:
# 株価情報が下降トレンドのパーフェクトオーダー（日足）かどうかの判断
filtered_reverse_perfect_order_week_day_tse_grows = filtered_ticker(filtered_reverse_perfect_order_week_tse_grows,  perfect_order={'mas': [5, 20, 60], 'reverse': True})
filtered_reverse_perfect_order_week_day_tse_grows

,銘柄名,コード,業種,出来高,株価
0,ＩＴｂｏｏｋホールディングス,1447,建設業,300200,311.0
1,メルディアＤＣ,1739,建設業,341600,794.0
2,総医研ホールディングス,2385,サービス業,223300,248.0
3,海帆,3133,小売業,1614200,396.0
4,プラスアルファ・コンサルティング,4071,情報・通信業,811800,2594.0
5,ＥＮＥＣＨＡＮＧＥ,4169,情報・通信業,547000,1040.0
6,ココナラ,4176,情報・通信業,345600,289.0
7,エクサウィザーズ,4259,情報・通信業,1282000,394.0
8,セキュア,4264,情報・通信業,236400,1628.0
9,エーアイ,4388,情報・通信業,313900,1065.0


In [32]:
# CSVファイル保存
save_df_as_csv(filtered_reverse_perfect_order_week_day_tse_grows, '6_東証G_DPO')

# 東証スタンダード・プライム選抜をCSVとして保存

In [33]:
import pandas as pd

def remove_duplicate_rows(dataframe, *other_dataframes):
    """重複しているティッカーコードの行を削除する

    :param dataframe: 対象のデータフレーム
    :param other_dataframes: 重複しているかどうかの判断のためのデータフレーム
    :return 引数dataframeで重複行を削除したもの
    """
    # データフレームをコピーして新しいデータフレームを作成
    result_df = dataframe.copy()

    # 重複確認のためのセットを作成
    unique_values = set()
    for df in other_dataframes:
        unique_values.update(df['コード'])

    # 重複しているティッカーコードを削除
    result_df = result_df[~result_df['コード'].isin(unique_values)]

    return result_df

In [34]:
# 東証スタンダードの銘柄コードを以下の条件に合致するもののみにフィルタリング
# 出来高　... 50万株以上
# 株価 ... １００円以上、5000円以下
filtered_tse_standard_2 = filtered_ticker(tse_standard, min_volume=500000, min_price=100, max_price=5000)
filtered_tse_standard_2

Not Found: No data found, symbol may be delisted
銘柄コード[2777]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[4327]の株価情報を取得できなかったため、スキップしました。
Not Found: No data found, symbol may be delisted
銘柄コード[5104]の株価情報を取得できなかったため、スキップしました。
Not Found: No data found, symbol may be delisted
銘柄コード[5999]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[6032]の株価情報を取得できなかったため、スキップしました。
Not Found: No data found, symbol may be delisted
銘柄コード[7961]の株価情報を取得できなかったため、スキップしました。
Not Found: No data found, symbol may be delisted
銘柄コード[8521]の株価情報を取得できなかったため、スキップしました。
Not Found: No data found, symbol may be delisted
銘柄コード[9318]の株価情報を取得できなかったため、スキップしました。
Not Found: No data found, symbol may be delisted
銘柄コード[9734]の株価情報を取得できなかったため、スキップしました。


,銘柄名,コード,業種,出来高,株価
0,住石ホールディングス,1514,鉱業,571800,341.000000
1,クルーズ,2138,情報・通信業,2412600,944.000000
2,ｎｍｓ ホールディングス,2162,サービス業,546500,394.000000
3,出前館,2484,情報・通信業,754100,403.000000
4,アップルインターナショナル,2788,卸売業,1196900,459.000000
5,ポラリス・ホールディングス,3010,サービス業,961100,175.000000
6,ユナイテッド・スーパーマーケット・ホールディングス,3222,小売業,542500,1102.000000
7,ＴＨＥグローバル社,3271,不動産業,650600,380.000000
8,フジタコーポレーション,3370,小売業,1686900,360.000000
9,ジー・スリーホールディングス,3647,情報・通信業,987300,166.000000


In [35]:
# ティッカーコードが重複している行を削除
filtered_tse_standard_2_remove_duplicate = remove_duplicate_rows(filtered_tse_standard_2, filtered_perfect_order_week_day_tse_standard, filtered_reverse_perfect_order_week_day_tse_standard)
filtered_tse_standard_2_remove_duplicate

,銘柄名,コード,業種,出来高,株価
0,住石ホールディングス,1514,鉱業,571800,341.000000
3,出前館,2484,情報・通信業,754100,403.000000
6,ユナイテッド・スーパーマーケット・ホールディングス,3222,小売業,542500,1102.000000
11,サイバーステップ,3810,情報・通信業,513800,394.000000
12,ショーケース,3909,情報・通信業,1326500,356.000000
15,ゲームカード・ジョイコホールディングス,6249,機械,517500,3390.000000
16,千代田化工建設,6366,建設業,1312800,365.000000
17,ＪＡＬＣＯホールディングス,6625,不動産業,744600,195.000000
19,アライドテレシスホールディングス,6835,電気機器,924900,104.000000
20,フェローテックホールディングス,6890,電気機器,1185000,2945.000000


In [36]:
# CSVファイル保存
save_df_as_csv(filtered_tse_standard_2_remove_duplicate, '7_東証S_選抜')

In [37]:
# 東証グロースの銘柄コードを以下の条件に合致するもののみにフィルタリング
# 出来高　... 50万株以上
# 株価 ... １００円以上、5000円以下
filtered_tse_grows_2 = filtered_ticker(tse_grows, min_volume=500000, min_price=100, max_price=5000)
filtered_tse_grows_2

Not Found: No data found, symbol may be delisted
銘柄コード[21649]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[3782]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[6067]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[6172]の株価情報を取得できなかったため、スキップしました。


,銘柄名,コード,業種,出来高,株価
0,ジーエヌアイグループ,2160,医薬品,5448900,1805.0
1,ウェッジホールディングス,2388,その他金融業,996900,134.0
2,アマナ,2402,サービス業,603200,362.0
3,海帆,3133,小売業,1614200,396.0
4,オークファン,3674,情報・通信業,1035600,490.0
5,Ａｉｍｉｎｇ,3911,情報・通信業,17011200,416.0
6,シェアリングテクノロジー,3989,情報・通信業,1297200,683.0
7,プラスアルファ・コンサルティング,4071,情報・通信業,811800,2594.0
8,プレイド,4165,情報・通信業,988100,617.0
9,ＥＮＥＣＨＡＮＧＥ,4169,情報・通信業,547000,1040.0


In [38]:
# ティッカーコードが重複している行を削除
filtered_tse_grows_2_remove_duplicate = remove_duplicate_rows(filtered_tse_grows_2, filtered_perfect_order_week_day_tse_grows, filtered_reverse_perfect_order_week_day_tse_grows)
filtered_tse_grows_2_remove_duplicate

,銘柄名,コード,業種,出来高,株価
1,ウェッジホールディングス,2388,その他金融業,996900,134.0
2,アマナ,2402,サービス業,603200,362.0
4,オークファン,3674,情報・通信業,1035600,490.0
6,シェアリングテクノロジー,3989,情報・通信業,1297200,683.0
8,プレイド,4165,情報・通信業,988100,617.0
17,カルナバイオサイエンス,4572,医薬品,832700,783.0
19,メドレックス,4586,医薬品,1863300,184.0
25,ＡＮＹＣＯＬＯＲ,5032,情報・通信業,1012600,3135.0
27,カバー,5253,情報・通信業,6465900,2190.0
30,Ｕｎｉｐｏｓ,6550,サービス業,4898300,225.0


In [39]:
# CSVファイル保存
save_df_as_csv(filtered_tse_grows_2_remove_duplicate, '8_東証G_選抜')